In [1]:
import data.dataset
import model.prompter
import utils.eval
import torch

In [2]:
it = data.dataset.get_data_loader("training", "naive_to_rgb_and_normalize", batch_size=1, shuffle=False, device = "cpu")

AssertionError: Invalid transform key!

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from segment_anything import SamPredictor, sam_model_registry

model_type = "vit_h"
model_checkpoint = "checkpoint/sam_vit_h_4b8939.pth"

sam = sam_model_registry[model_type](checkpoint=model_checkpoint)
sam_predictor = SamPredictor(sam)

Promptor = model.prompter.Prompter()


In [4]:
import matplotlib.pyplot as plt
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)  

def predict(image, label):

    global image2
    image2 = image.numpy()


    sam_predictor.set_image(image)
    result = np.zeros(label.shape)
    scores = np.zeros(label.shape, dtype = float)
    

    for i in range(1, 14):
        #print(i)
        mask = np.zeros(label.shape, dtype = np.uint8)
        mask[label == i] = 1
        if (np.max(mask) == 0):
            continue
        prompt = Promptor(mask, 1)
        #for j in range(mask.shape[0]):
        #    for k in range(mask.shape[1]):
        #        print(mask[j][k], end = ' ')
        #    print()
        #print(prompt)
        msk, score, logit = sam_predictor.predict(**prompt)  

        x, y, z = msk.shape
        msk = msk.reshape(y, z)

        # max_id = 1
        # print(score)

        plt.ion()

        plt.figure(figsize=(10,10))
        plt.imshow(image)
        show_points(prompt['point_coords'], prompt['point_labels'], plt.gca())
        plt.axis('on')
        plt.show()  


        cond = (scores < score[0]) & msk
        scores[cond] = score[0]
        result[cond] = i

    global res2
    res2 = result

    global ulabel
    ulabel = label.numpy()

    return result


In [5]:
print(utils.eval.evaluate(predict, it))

tensor([[[-2.1179, -2.1179, -2.1179,  ...,  0.0000,  0.0000,  0.0000],
         [-2.1179, -2.1179, -2.1179,  ...,  0.0000,  0.0000,  0.0000],
         [-2.1179, -2.1179, -2.1179,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-2.1179, -2.1179, -2.1179,  ...,  0.0000,  0.0000,  0.0000],
         [-2.1179, -2.1179, -2.1179,  ...,  0.0000,  0.0000,  0.0000],
         [-2.1179, -2.1179, -2.1179,  ...,  0.0000,  0.0000,  0.0000]],

        [[-2.0357, -2.0357, -2.0357,  ...,  0.0000,  0.0000,  0.0000],
         [-2.0357, -2.0357, -2.0357,  ...,  0.0000,  0.0000,  0.0000],
         [-2.0357, -2.0357, -2.0357,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-2.0357, -2.0357, -2.0357,  ...,  0.0000,  0.0000,  0.0000],
         [-2.0357, -2.0357, -2.0357,  ...,  0.0000,  0.0000,  0.0000],
         [-2.0357, -2.0357, -2.0357,  ...,  0.0000,  0.0000,  0.0000]],

        [[-1.8044, -1.8044, -1.8044,  ...,  0.0000,  0.0000,  0.0000],
         [-1.8044, -1.8044, -1.8044,  ...,  0

c:\Users\32427\Documents\GitHub\ML_Project\utils\eval.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cur = dice(torch.tensor(val_outputs, dtype = torch.int), torch.tensor(val_labels[0][0], dtype = torch.int))


In [6]:
x, y = res2.shape
print(x, y)

print(image2)

image3 = ((ulabel == 0)).reshape(1, x, y) * image2 + ((ulabel != 0).reshape(1, x, y) * np.array([[[1]], [[0]], [[0]]], dtype = float) + image2) / 2

print(image3)
cv2.imshow("result", image3)
cv2.waitKey(0)

1024 1024
[[[-2.117904  -2.117904  -2.117904  ...  0.         0.         0.       ]
  [-2.117904  -2.117904  -2.117904  ...  0.         0.         0.       ]
  [-2.117904  -2.117904  -2.117904  ...  0.         0.         0.       ]
  ...
  [-2.117904  -2.117904  -2.117904  ...  0.         0.         0.       ]
  [-2.117904  -2.117904  -2.117904  ...  0.         0.         0.       ]
  [-2.117904  -2.117904  -2.117904  ...  0.         0.         0.       ]]

 [[-2.0357144 -2.0357144 -2.0357144 ...  0.         0.         0.       ]
  [-2.0357144 -2.0357144 -2.0357144 ...  0.         0.         0.       ]
  [-2.0357144 -2.0357144 -2.0357144 ...  0.         0.         0.       ]
  ...
  [-2.0357144 -2.0357144 -2.0357144 ...  0.         0.         0.       ]
  [-2.0357144 -2.0357144 -2.0357144 ...  0.         0.         0.       ]
  [-2.0357144 -2.0357144 -2.0357144 ...  0.         0.         0.       ]]

 [[-1.8044444 -1.8044444 -1.8044444 ...  0.         0.         0.       ]
  [-1.804444

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:124: error: (-215:Assertion failed) bmi && width >= 0 && height >= 0 && (bpp == 8 || bpp == 24 || bpp == 32) in function 'FillBitmapInfo'
